In [7]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout
import sklearn
from sklearn.model_selection import train_test_split
import os
from os import listdir
from zipfile import ZipFile
import pickle
import cv2
import gdown
from tensorflow.keras.utils import img_to_array
import numpy as np
import sklearn
from sklearn.preprocessing import LabelBinarizer

In [2]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            img = cv2.resize(image, (32,32))
            return img_to_array(img)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [9]:
def load_dataset(dir):
    ImgList, label_list = [], []
    try:
            print("Load Dataset -->")
            root_dir = listdir(dir)
            print("Root Directory : ", root_dir)
            for shape in root_dir:
                if shape == ".DS_Store":
                    root_dir.remove(dir)
            
            for shape in root_dir:
                faceShape_image_list = listdir(f"{dir}/{shape}")
                
                for faceShape in faceShape_image_list:
                    
                    if faceShape == ".DS_Store":
                        faceShape_image_list.remove(faceShape)
                        
                for faceShape_image in faceShape_image_list[:7000]:
                    
                    img_dir = f"{root_dir}/{shape}/{faceShape_image}"
                    
                    if img_dir.endswith(".jpg") == True or img_dir.endswith(".JPG") == True or img_dir.endswith(".png") == True or img_dir.endswith(".PNG") == True:
                        
                        converted_image = convert_image_to_array(img_dir)
                        ImgList.append(converted_image)
                        label_list.append(shape)
            print("Loading completed")
            return ImgList, label_list
    except Exception as e:
        print(f"Error : {e}")
        

In [10]:
images, labels = load_dataset("faceshape-master/published_dataset")

Load Dataset -->
Root Directory :  ['heart', 'oblong', 'oval', 'round', 'square']
Loading completed


In [11]:
print(len(images))

500


In [12]:
label_gen = LabelBinarizer()
image_labels = label_gen.fit_transform(labels)
pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
n_classes = len(label_gen.classes_)
print(n_classes)
print(image_labels)

5
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [13]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
x_train, x_test, y_train, y_test = train_test_split(images, image_labels, test_size=0.3, shuffle=True,random_state = 42) 
print(x_train.shape)
print(x_test.shape)

making the training and testing split with 30% of the dataset as testing set


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# cnn model 

# Define the input shape of the images
input_shape = (32, 32, 3)

# Initialize the model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the flatten layer
model.add(Flatten())

# Add the dense layer
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model architecture
model.summary()

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=16)


In [ ]:
# Evaluation 

y_pred = model.predict(x_orig_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

# Calculate the true labels for the test set
y_true = y_orig_test

accuracy = accuracy_score(y_orig_test, y_pred_binary)
precision = precision_score(y_orig_test, y_pred_binary)
recall = recall_score(y_orig_test, y_pred_binary)
f1 = f1_score(y_orig_test, y_pred_binary)
conf_matrix = confusion_matrix(y_orig_test, y_pred_binary)


print("Accuracy:", accuracy * 100)
print("Precision:", precision * 100)
print("Recall:", recall * 100)
print("F1-score:", f1 * 100)